In [0]:
from pyspark.sql import functions as F

silver = "/Volumes/ecommerce_cat/ecommerce_schema/silver"
gold   = "/Volumes/ecommerce_cat/ecommerce_schema/gold"

In [0]:
df = spark.read.format("delta").load(f"{silver}/order_reviews")

fact_reviews = df.select(
    "review_id",
    "order_id",
    "review_score",
    "review_creation_date"
)


In [0]:
fact_reviews.write.format("delta").mode("overwrite").save(f"{gold}/fact_reviews")

In [0]:
%sql
CREATE OR REPLACE VIEW ecommerce_cat.ecommerce_schema.fact_reviews AS
SELECT * FROM delta.`/Volumes/ecommerce_cat/ecommerce_schema/gold/fact_reviews`;

In [0]:
%sql
CREATE OR REPLACE TABLE ecommerce_cat.ecommerce_schema.gold_reviews AS
SELECT
    review_id,
    order_id,

    -- Coluna original
    review_score,

    -- Coluna limpa usando TRY_CAST
    TRY_CAST(review_score AS INT) AS review_score_clean,

    review_creation_date

FROM ecommerce_cat.ecommerce_schema.fact_reviews;


In [0]:
%sql
select * from ecommerce_cat.ecommerce_schema.gold_reviews limit 10